In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
huggingface_token = os.getenv("HF_TOKEN_read")

from huggingface_hub import login
login(token=huggingface_token)

In [2]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
    "../../Ophtimus/Ophtimus-3B-Instruct-v1/4epoch",
    torch_dtype=torch.bfloat16,
    device_map = {"": 0}
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.add_special_tokens({"pad_token": "<|finetune_right_pad_id|>"})
model.config.pad_token_id = tokenizer.pad_token_id # 128004

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import AutoTokenizer
import torch

from datasets import load_dataset, concatenate_datasets

MCQA_dataset = load_dataset("BaekSeungJu/PubMedQA-Ophthal-Dataset", split="train")
MCQA_dataset = MCQA_dataset.shuffle(seed=42)

def MCQA_format_chat_template(row):
    # System instruction
    system_instruction = (
        "You are an expert ophthalmologist. Please provide accurate and "
        "medically sound answers to the user's ophthalmology-related question."
    )
    
    def MCQA_Instruction_formatting_func(question, option_a, option_b, option_c, option_d, option_e):
        """Format the question and the available answer choices."""
        # Check how many valid options there are to display
        if option_c == "":
            # Only A and B
            return (f"Question:\n{question}\n\nOptions:\n"
                    f"A) {option_a}\nB) {option_b}")
        elif option_e == "":
            # Options A, B, C, D
            return (f"Question:\n{question}\n\nOptions:\n"
                    f"A) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}")
        else:
            # Options A, B, C, D, E
            return (f"Question:\n{question}\n\nOptions:\n"
                    f"A) {option_a}\nB) {option_b}\nC) {option_c}\nD) {option_d}\nE) {option_e}")
    
    # 4. Build the chat template JSON
    row_json = [
        {"role": "system", "content": system_instruction},
        {
            "role": "user",
            "content": MCQA_Instruction_formatting_func(
                row["question"],
                "True",
                "False",
                "",
                "",
                ""
            )
        }
        
    ]
    
    # 5. Convert or tokenize the row_json however you like
    row["text"] = tokenizer.apply_chat_template(row_json, add_generation_prompt=True, tokenize=False)
    
    return row

Eval_Dataset = MCQA_dataset.map(
    MCQA_format_chat_template,
    num_proc= 4,
)

print(f"Test dataset size: {len(Eval_Dataset)}")

Test dataset size: 297


In [4]:
def test_model(prompt, model):
    inputs = tokenizer(
    [
        prompt
    ], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = False, temperature = 0.0005, top_p = 0.95)
    return(tokenizer.batch_decode(outputs)[0])

In [5]:
import re

answer_list = []
for idx in range(len(Eval_Dataset)):
    if idx%50==0:
        print(f"{idx} / {len(Eval_Dataset)}")
    model_answer = test_model(Eval_Dataset[idx]["text"] ,model=model)
    
    answer_ = re.search(
    r"<\|start_header_id\|>assistant<\|end_header_id\|>\n\n(.*?)(<\|eot_id\|>|$)",
    model_answer, re.DOTALL).group(1)
    
    answer_list.append(answer_)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


0 / 297


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

50 / 297


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

200 / 297


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

250 / 297


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [6]:
import pandas as pd

model_name = "Ophtimus-3B-Instruct-v1-4epoch"
df = pd.DataFrame(answer_list, columns=['answer'])
df.to_excel(f'./Dataset/PubMedQAEval_{model_name}.xlsx', index=False)